# RoPE + YaRN

目标：
- 理解 RoPE 的旋转编码与“相对位置信息”的本质
- 理解 YaRN 如何在不改模型参数的情况下扩展可用上下文
- 用本项目实现（`precompute_freqs_cis` / `apply_rotary_pos_emb`）验证概念


In [2]:
# 基础依赖
import math
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt

# 使用项目里的 RoPE 实现
from vermind_models.base_module import precompute_freqs_cis, apply_rotary_pos_emb
from vermind_models.config.config import VerMindConfig

pd.set_option("display.max_rows", 10)
plt.rcParams["figure.figsize"] = (8, 4)


In [ ]:
# Hook：直接展示 RoPE 的使用位置（而不是手动翻文件）
import inspect
import textwrap
from vermind_models import base_module as bm
from vermind_models import GQA as gqa

print("[use] apply_rotary_pos_emb in GQA.MergedGQA.forward:
")
print(textwrap.dedent(inspect.getsource(gqa.MergedGQA.forward)))

print("
[impl] apply_rotary_pos_emb:
")
print(textwrap.dedent(inspect.getsource(bm.apply_rotary_pos_emb)))

print("
[impl] precompute_freqs_cis (RoPE+YaRN):
")
print(textwrap.dedent(inspect.getsource(bm.precompute_freqs_cis)))


## 1. RoPE 的核心思想：把位置编码成“旋转”

RoPE 的关键是：把每两个维度视为一个 2D 平面上的向量，然后按位置 `pos` 旋转一个角度 `θ(pos)`。

- 对第 `i` 个 2D 维度对，旋转角度是：

$$
\theta_i(pos) = pos \cdot \omega_i
$$

- 频率 $$\omega_i$$ 由维度索引决定：

$$
\omega_i = \frac{1}{\text{rope\_base}^{\frac{2i}{d}}}
$$

把每个 2D 向量 $$(x_1, x_2)$$ 做旋转：

$$
\begin{bmatrix}x_1' \\ x_2'\end{bmatrix} =
\begin{bmatrix}\cos\theta & -\sin\theta \\ \sin\theta & \cos\theta\end{bmatrix}
\begin{bmatrix}x_1 \\ x_2\end{bmatrix}
$$

在源码里，这对应 `apply_rotary_pos_emb` 中的 `rotate_half` + 组合 cos/sin 的写法。


## 2. 源码如何实现 RoPE（`precompute_freqs_cis`）

查看 `vermind_models/base_module.py`：

- `precompute_freqs_cis` 先计算 `inv_freq = 1 / rope_base^(2i/d)`
- 再做 `t = [0..end)` 与 `inv_freq` 的外积，得到每个位置的角度
- 最终输出 `cos`/`sin` 缓存

这个函数还支持 `rope_scaling`，即 **YaRN** 的频率缩放（后面详解）。


In [ ]:
# 对照本项目默认配置
cfg = VerMindConfig()
head_dim = cfg.hidden_size // cfg.num_attention_heads

cfg.rope_theta, head_dim


In [ ]:
# 计算 RoPE 频率（未启用 YaRN）
dim = head_dim  # rotary dim
rope_base = cfg.rope_theta

inv_freq = 1.0 / (rope_base ** (torch.arange(0, dim, 2).float() / dim))

# 用 pandas 看看前几维的频率
pd.DataFrame({"dim_index": torch.arange(0, dim, 2), "inv_freq": inv_freq}).head(6)


## 3. YaRN 的直观解释：在“高频维度”上减速

YaRN 目标是：在不改模型参数的情况下，把最大有效上下文 **拉长**。

本项目的实现位置：`precompute_freqs_cis`。

关键步骤（源码逻辑）：
- 计算 `low/high` 两个维度边界（由 `beta_fast` / `beta_slow` 控制）
- 在 `low~high` 区间做线性 ramp
- 用 `freqs = freqs * (1 - ramp + ramp / factor)` 拉低高频

这样做的结果：高维（高频）部分的“旋转速度”变慢，能覆盖更长的相对距离。


In [ ]:
# 用源码同样的方式计算 YaRN ramp 和缩放
rope_scaling = {
    "beta_fast": 32,
    "beta_slow": 1,
    "factor": 16,
    "original_max_position_embeddings": 2048,
    "attention_factor": 1.0,
}

orig_max = rope_scaling["original_max_position_embeddings"]
factor = rope_scaling["factor"]
beta_fast = rope_scaling["beta_fast"]
beta_slow = rope_scaling["beta_slow"]

# 按源码的 inv_dim 公式
inv_dim = lambda b: (dim * math.log(orig_max / (b * 2 * math.pi))) / (2 * math.log(rope_base))
low = max(math.floor(inv_dim(beta_fast)), 0)
high = min(math.ceil(inv_dim(beta_slow)), dim // 2 - 1)

ramp = torch.clamp((torch.arange(dim // 2).float() - low) / max(high - low, 0.001), 0, 1)
scale = (1 - ramp + ramp / factor)

pd.DataFrame({
    "pair_index": torch.arange(dim // 2),
    "ramp": ramp,
    "scale": scale,
}).head(8)


In [ ]:
# pandas 绘图：不同维度对上的缩放系数
scale_df = pd.DataFrame({
    "pair_index": torch.arange(dim // 2).numpy(),
    "scale": scale.numpy(),
})

ax = scale_df.plot(x="pair_index", y="scale", title="YaRN 缩放系数（按维度对）", legend=False)
ax.set_xlabel("维度对索引")
ax.set_ylabel("scale")
plt.show()


In [ ]:
# 对比：原始 inv_freq vs YaRN 缩放后的 inv_freq
inv_freq_yarn = inv_freq * scale

freq_df = pd.DataFrame({
    "pair_index": torch.arange(dim // 2).numpy(),
    "inv_freq": inv_freq.numpy(),
    "inv_freq_yarn": inv_freq_yarn.numpy(),
})

ax = freq_df.plot(x="pair_index", y=["inv_freq", "inv_freq_yarn"], title="RoPE 频率对比：原始 vs YaRN")
ax.set_xlabel("维度对索引")
ax.set_ylabel("inv_freq")
plt.show()


## 4. RoPE 的相对位置性质（用源码验证）

RoPE 的一个重要特性：
> 经过旋转后的 `q` 和 `k` 的点积依赖于 **相对位置差**。

下面用 `apply_rotary_pos_emb` 做一个小实验：固定一个 token 的向量，改变另一个 token 的位置，观察相似度随位置变化。


In [ ]:
# 构造一个简单的 q/k 例子
torch.manual_seed(42)
seq_len = 64
bsz = 1
n_heads = 1

# q/k 形状：[bs, seq_len, n_heads, head_dim]
q = torch.randn(bsz, seq_len, n_heads, dim)
k = torch.randn(bsz, seq_len, n_heads, dim)

# 预计算 cos/sin
cos, sin = precompute_freqs_cis(dim=dim, end=seq_len, rope_base=rope_base, rope_scaling=None)

# 选定一个 query 位置，对比不同 key 位置的相似度
q_pos = 10

# 应用 RoPE
q_rot, k_rot = apply_rotary_pos_emb(q, k, cos, sin)

# 计算 dot(q_pos, k_pos)
q_vec = q_rot[:, q_pos, 0, :]

dots = []
for k_pos in range(seq_len):
    k_vec = k_rot[:, k_pos, 0, :]
    dots.append(torch.sum(q_vec * k_vec, dim=-1).item())

sim_df = pd.DataFrame({"k_pos": np.arange(seq_len), "dot": dots})
ax = sim_df.plot(x="k_pos", y="dot", title=f"RoPE 后的相似度曲线（q_pos={q_pos}）", legend=False)
ax.set_xlabel("k_pos")
ax.set_ylabel("dot")
plt.show()


## 5. 源码中的 YaRN 配置入口

`vermind_models/config/config.py` 中：
- `rope_theta`：RoPE 基频
- `inference_rope_scaling=True` 时启用 `rope_scaling`（类型为 `yarn`）

这会影响 `precompute_freqs_cis(..., rope_scaling=cfg.rope_scaling)` 的频率缓存。


In [ ]:
# 展示 VerMindConfig 中的 YaRN 配置
cfg = VerMindConfig(inference_rope_scaling=True)

cfg.rope_theta, cfg.rope_scaling


## 6. 总结（对应源码关键点）

- `precompute_freqs_cis`：
  - 负责 `inv_freq` 与 `cos/sin` 的预计算
  - 当 `rope_scaling` 存在时，用 YaRN 公式调节频率

- `apply_rotary_pos_emb`：
  - 用 `rotate_half` + `cos/sin` 完成二维旋转
  - 支持 `position_ids` 与连续序列两种路径

- `VerMindConfig`：
  - 控制 `rope_theta` 与 YaRN 超参
  - 推理阶段可通过 `inference_rope_scaling=True` 延长上下文

如果你希望，我可以继续补充：
- 更贴近论文公式的推导
- 对比 NTK / linear scaling 与 YaRN 的差异
- 基于你自己的 checkpoint 做可视化分析
